# Downloading dataset 

In [6]:
import datasets
from datasets import load_dataset

In [7]:
# Get user's home directory
import os
home = os.path.expanduser("~")

# Define the path of the cache directory
cache_dir = os.path.join(home, ".cache", "huggingface", "datasets")

# Define the name and configuration of the dataset
dataset_name = "wmt14"
config_name = "fr-en"

# Build the path for the specific dataset configuration
dataset_config_path = os.path.join(cache_dir, dataset_name, config_name)

print(f"Checking cache at: {dataset_config_path}")

# Check if the dataset configuration is already cached
if os.path.exists(dataset_config_path) and len(os.listdir(dataset_config_path)) > 0:
    print("Dataset already downloaded, loading from cache.")
    # If the dataset is already downloaded, load it from the cache directory
    dataset = load_dataset(dataset_name, config_name, cache_dir=cache_dir)
else:
    print("Downloading the dataset.")
    # Download the dataset and specify the cache directory
    dataset = load_dataset(dataset_name, config_name, cache_dir=cache_dir)

# Here, you should adjust the loading of subsets to avoid redundant downloads or loading.
# Load 50k rows of the train dataset
train_dataset = dataset["train"].select(range(100020))
# train_dataset = dataset["train"].select(range(600))

# Keep the full valid and test datasets
valid_dataset = dataset["validation"]
test_dataset = dataset["test"]

Checking cache at: /root/.cache/huggingface/datasets/wmt14/fr-en
Dataset already downloaded, loading from cache.


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
test_dataset

Dataset({
    features: ['translation'],
    num_rows: 3003
})

In [9]:
texts =[]
labels = []
for element in test_dataset["translation"]:
        # print("element: ", element)
        texts.append(element["en"])
        labels.append(element["fr"])

In [24]:
texts

['Spectacular Wingsuit Jump Over Bogota',
 'Sportsman Jhonathan Florez jumped from a helicopter above Bogota, the capital of Colombia, on Thursday.',
 'Wearing a wingsuit, he flew past over the famous Monserrate Sanctuary at 160km/h. The sanctuary is located at an altitude of over 3000 meters and numerous spectators had gathered there to watch his exploit.',
 'A black box in your car?',
 "As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.",
 "The devices, which track every mile a motorist drives and transmit that information to bureaucrats, are at the center of a controversial attempt in Washington and state planning offices to overhaul the outdated system for funding America's major roads.",
 'The usually dull arena of highway planning has suddenly spawned intense debate and colorful alliances.',
 'Libertarians have joined environmental groups

In [10]:
import evaluate
metric = evaluate.load("sacrebleu")
import os
getpwd = os.getcwd()

In [6]:
# Specify the file path

file_path_en = os.path.join(getpwd, "original_english.txt")
# file_path = "/path/to/translations.txt"

# Open the file in write mode
with open(file_path_en, "w") as file:
    # Write each translation to the file
    for text in texts:
        file.write(text + "\n")

In [7]:
# Specify the file path
# import os
# getpwd = os.getcwd()
file_path_fr = os.path.join(getpwd, "original_french.txt")
# file_path = "/path/to/translations.txt"

# Open the file in write mode
with open(file_path_fr, "w") as file:
    # Write each translation to the file
    for label in labels:
        file.write(label + "\n")

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

# google-t5/t5-small

In [17]:
# # Generate multiple translations in batched format


# # Initialize the tokenizer and model
tokenizer_t5_small_pretrained = AutoTokenizer.from_pretrained("google-t5/t5-small")
model_t5_small_pretrained = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

translations_t5_small_pretrained = []
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("Original English Sentence: ", text)
    # print("Original French Sentence: ", labels[idx])
    inputs_t5_small_pretrained = tokenizer_t5_small_pretrained(text, return_tensors="pt").input_ids
    outputs_t5_small_pretrained = model_t5_small_pretrained.generate(inputs_t5_small_pretrained, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    translation_t5_small_pretrained = tokenizer_t5_small_pretrained.decode(outputs_t5_small_pretrained[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_small_pretrained)
    translations_t5_small_pretrained.append(translation_t5_small_pretrained)
    # print("\n")

# Specify the file path
file_path_t5_small_pretrained = os.path.join(getpwd, "translated_french_by_t5_small_pretrained.txt")

# Open the file in write mode
with open(file_path_t5_small_pretrained, "w") as file:
    # Write each translation to the file
    for translation in translations_t5_small_pretrained:
        file.write(translation + "\n")


Translating: 100%|██████████| 3003/3003 [23:23<00:00,  2.14it/s]


In [18]:
result_t5_small_pretrained = metric.compute(predictions=translations_t5_small_pretrained, references=labels)
result_t5_small_pretrained = {"bleu": result_t5_small_pretrained["score"]}
result_t5_small_pretrained

{'bleu': 2.041124318007492}

# google-t5/t5-base

In [18]:
# # Initialize the tokenizer and model
tokenizer_t5_base_pretrained = AutoTokenizer.from_pretrained("google-t5/t5-base")
model_t5_base_pretrained = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-base")

translations_t5_base_pretrained = []
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("Original English Sentence: ", text)
    # print("Original French Sentence: ", labels[idx])
    inputs_t5_base_pretrained = tokenizer_t5_base_pretrained(text, return_tensors="pt").input_ids
    outputs_t5_base_pretrained = model_t5_base_pretrained.generate(inputs_t5_base_pretrained, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    translation_t5_base_pretrained = tokenizer_t5_base_pretrained.decode(outputs_t5_base_pretrained[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_small_pretrained)
    translations_t5_base_pretrained.append(translation_t5_base_pretrained)
    # print("\n")

# Specify the file path
file_path_t5_base_pretrained = os.path.join(getpwd, "translated_french_by_t5_base_pretrained.txt")

# Open the file in write mode
with open(file_path_t5_base_pretrained, "w") as file:
    # Write each translation to the file
    for translation in translations_t5_base_pretrained:
        file.write(translation + "\n")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Translating: 100%|██████████| 3003/3003 [50:13<00:00,  1.00s/it] 


In [21]:
result_t5_base_pretrained = metric.compute(predictions=translations_t5_base_pretrained, references=labels)
result_t5_base_pretrained = {"bleu": result_t5_base_pretrained["score"]}
result_t5_base_pretrained

{'bleu': 1.196346176518329}

# Helsinki-NLP/opus-mt-en-fr

In [16]:
# # Initialize the tokenizer and model
tokenizer_opus_mt_en_fr_pretrained = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model_opus_mt_en_fr_pretrained = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

translations_opus_mt_en_fr_pretrained = []
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("Original English Sentence: ", text)
    # print("Original French Sentence: ", labels[idx])
    inputs_opus_mt_en_fr_pretrained= tokenizer_opus_mt_en_fr_pretrained(text, return_tensors="pt").input_ids
    outputs_opus_mt_en_fr_pretrained = model_opus_mt_en_fr_pretrained.generate(inputs_opus_mt_en_fr_pretrained, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    translation_opus_mt_en_fr_pretrained = tokenizer_opus_mt_en_fr_pretrained.decode(outputs_opus_mt_en_fr_pretrained[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_small_pretrained)
    translations_opus_mt_en_fr_pretrained.append(translation_opus_mt_en_fr_pretrained)
    # print("\n")

# Specify the file path
file_path_opus_mt_en_fr_pretrained = os.path.join(getpwd, "translated_french_by_opus_mt_en_fr_pretrained.txt")

# Open the file in write mode
with open(file_path_opus_mt_en_fr_pretrained, "w") as file:
    # Write each translation to the file
    for translation in translations_opus_mt_en_fr_pretrained:
        file.write(translation + "\n")

Translating: 100%|██████████| 3003/3003 [55:23<00:00,  1.11s/it]  


In [17]:
result_opus_mt_en_fr_pretrained = metric.compute(predictions=translations_opus_mt_en_fr_pretrained, references=labels)
result_opus_mt_en_fr_pretrained = {"bleu": result_opus_mt_en_fr_pretrained["score"]}
result_opus_mt_en_fr_pretrained

{'bleu': 33.13527604394126}

# sriram-sanjeev9s/T5_wmt14_En_Fr_1million

In [19]:
tokenizer_t5_finetuned_wmt14_1mil = AutoTokenizer.from_pretrained("sriram-sanjeev9s/T5_wmt14_En_Fr_1million")
model_t5_finetuned_wmt14_1mil = AutoModelForSeq2SeqLM.from_pretrained("sriram-sanjeev9s/T5_wmt14_En_Fr_1million")

translations_t5_finetuned_wmt14_1mil = []
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("english sentence: ", text)
    # print("Original French Sentence: ", labels[texts.index(text)])
    inputs_t5_finetuned_wmt14_1mil = tokenizer_t5_finetuned_wmt14_1mil(text, return_tensors="pt").input_ids
    # outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    outputs_t5_finetuned_wmt14_1mil = model_t5_finetuned_wmt14_1mil.generate(inputs_t5_finetuned_wmt14_1mil, max_length=60, num_beams=5, early_stopping=True)
    translation_t5_finetuned_wmt14_1mil = tokenizer_t5_finetuned_wmt14_1mil.decode(outputs_t5_finetuned_wmt14_1mil[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_finetuned_wmt14_1mil)
    translations_t5_finetuned_wmt14_1mil.append(translation_t5_finetuned_wmt14_1mil)
    # print("\n")

# Specify the file path
file_path_t5_finetuned_wmt14_1mil = os.path.join(getpwd, "translated_french_by_t5_finetuned_wmt14_1mil.txt")

# Open the file in write mode
with open(file_path_t5_finetuned_wmt14_1mil, "w") as file:
    # Write each translation to the file
    for translation in translations_t5_finetuned_wmt14_1mil:
        file.write(translation + "\n")


Translating: 100%|██████████| 3003/3003 [38:09<00:00,  1.31it/s]


# ######################################

In [21]:
result_t5_finetuned_wmt14_1mil = metric.compute(predictions=translations_t5_finetuned_wmt14_1mil, references=labels)
result_t5_finetuned_wmt14_1mil = {"bleu": result_t5_finetuned_wmt14_1mil["score"]}
result_t5_finetuned_wmt14_1mil

{'bleu': 21.71904404108645}

# sriram-sanjeev9s/opus-mt-en-fr_wmt14_En_Fr_1million_20epochs

In [11]:
tokenizer_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = AutoTokenizer.from_pretrained("sriram-sanjeev9s/opus-mt-en-fr-finetuned-en-to-fr")
model_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = AutoModelForSeq2SeqLM.from_pretrained("sriram-sanjeev9s/opus-mt-en-fr-finetuned-en-to-fr")

translations_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = []
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("english sentence: ", text)
    # print("Original French Sentence: ", labels[texts.index(text)])
    inputs_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = tokenizer_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs(text, return_tensors="pt").input_ids
    # outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    outputs_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = model_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs.generate(inputs_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs, max_length=60, num_beams=5, early_stopping=True)
    translation_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = tokenizer_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs.decode(outputs_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_finetuned_wmt14_1mil)
    translations_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs.append(translation_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs)
    # print("\n")

# Specify the file path
file_path_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = os.path.join(getpwd, "translated_french_by_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs.txt")

# Open the file in write mode
with open(file_path_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs, "w") as file:
    # Write each translation to the file
    for translation in translations_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs:
        file.write(translation + "\n")

tokenizer_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/299M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

Translating: 100%|██████████| 3003/3003 [17:56<00:00,  2.79it/s]


In [12]:
result_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = metric.compute(predictions=translations_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs, references=labels)
result_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs = {"bleu": result_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs["score"]}
result_opus_mt_en_fr_wmt14_En_Fr_1million_20epochs

{'bleu': 33.84192310779139}

# sriram-sanjeev9s/T5_base_wmt14_En_Fr_1million

In [24]:
tokenizer_t5_base_finetuned_wmt14_1mil = AutoTokenizer.from_pretrained("sriram-sanjeev9s/T5_base_wmt14_En_Fr_1million")
model_t5_base_finetuned_wmt14_1mil = AutoModelForSeq2SeqLM.from_pretrained("sriram-sanjeev9s/T5_base_wmt14_En_Fr_1million")

translations_t5_base_finetuned_wmt14_1mil = []
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("english sentence: ", text)
    # print("Original French Sentence: ", labels[texts.index(text)])
    inputs_t5_base_finetuned_wmt14_1mil = tokenizer_t5_base_finetuned_wmt14_1mil(text, return_tensors="pt").input_ids
    # outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    outputs_t5_base_finetuned_wmt14_1mil = model_t5_base_finetuned_wmt14_1mil.generate(inputs_t5_base_finetuned_wmt14_1mil, max_length=60, num_beams=5, early_stopping=True)
    translation_t5_base_finetuned_wmt14_1mil = tokenizer_t5_base_finetuned_wmt14_1mil.decode(outputs_t5_base_finetuned_wmt14_1mil[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_finetuned_wmt14_1mil)
    translations_t5_base_finetuned_wmt14_1mil.append(translation_t5_base_finetuned_wmt14_1mil)
    # print("\n")

# Specify the file path
file_path_t5_base_finetuned_wmt14_1mil = os.path.join(getpwd, "translated_french_by_t5_base_finetuned_wmt14_1mil.txt")

# Open the file in write mode
with open(file_path_t5_base_finetuned_wmt14_1mil, "w") as file:
    # Write each translation to the file
    for translation in translations_t5_base_finetuned_wmt14_1mil:
        file.write(translation + "\n")

Translating: 100%|██████████| 3003/3003 [1:29:14<00:00,  1.78s/it]


In [25]:
result_t5_base_finetuned_wmt14_1mil = metric.compute(predictions=translations_t5_base_finetuned_wmt14_1mil, references=labels)
result_t5_base_finetuned_wmt14_1mil = {"bleu": result_t5_base_finetuned_wmt14_1mil["score"]}
result_t5_base_finetuned_wmt14_1mil

{'bleu': 23.78058757171012}

# saved generator from joint_train_Bert_dualG_v2_logTrnsDB_pg_kd.py wmt14_en_fr_800sent_pg_kd_loss

In [ ]:
tokenizer_t5_base_finetuned_wmt14_1mil = AutoTokenizer.from_pretrained("/home/paperspace/google_drive_v4/Research_Thesis/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_800sent_pg_kd_loss/best_generator.pt")
model_t5_base_finetuned_wmt14_1mil = AutoModelForSeq2SeqLM.from_pretrained("/home/paperspace/google_drive_v4/Research_Thesis/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_800sent_pg_kd_loss/best_generator.pt")


In [13]:
tokenizer_t5_base_finetuned_wmt14_1mil = AutoTokenizer.from_pretrained("/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/train_checkpoint_2.pt")
model_t5_base_finetuned_wmt14_1mil = AutoModelForSeq2SeqLM.from_pretrained("/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/train_checkpoint_2.pt")

translations_t5_base_finetuned_wmt14_1mil = []
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("english sentence: ", text)
    # print("Original French Sentence: ", labels[texts.index(text)])
    inputs_t5_base_finetuned_wmt14_1mil = tokenizer_t5_base_finetuned_wmt14_1mil(text, return_tensors="pt").input_ids
    # outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    outputs_t5_base_finetuned_wmt14_1mil = model_t5_base_finetuned_wmt14_1mil.generate(inputs_t5_base_finetuned_wmt14_1mil, max_length=60, num_beams=5, early_stopping=True)
    translation_t5_base_finetuned_wmt14_1mil = tokenizer_t5_base_finetuned_wmt14_1mil.decode(outputs_t5_base_finetuned_wmt14_1mil[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_finetuned_wmt14_1mil)
    translations_t5_base_finetuned_wmt14_1mil.append(translation_t5_base_finetuned_wmt14_1mil)
    # print("\n")

# Specify the file path
file_path_t5_base_finetuned_wmt14_1mil = os.path.join(getpwd, "translated_french_by_t5_base_finetuned_wmt14_1mil.txt")

# Open the file in write mode
with open(file_path_t5_base_finetuned_wmt14_1mil, "w") as file:
    # Write each translation to the file
    for translation in translations_t5_base_finetuned_wmt14_1mil:
        file.write(translation + "\n")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/train_checkpoint_2.pt'. Use `repo_type` argument if needed.

# Finetuned MarianMT in GAN 

In [ ]:
import torch

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
generator2_train = AutoModelForSeq2SeqLM.from_pretrained("/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/train_checkpoint_2.pt")

In [ ]:
model_path = '/root/Adversarial_NMT_th/checkpoints/joint/test_vastai_wmt14_en_fr_2023_24mil_to_32mil_8mgpu_3070Dloss_fullDict_v3/train_joint_g_10.497.epoch_1.pt'
assert os.path.exists(g_model_path)
generator = TransformerModel_custom(args, 
            dataset.src_dict, dataset.dst_dict, use_cuda=use_cuda)    
model_dict = generator.state_dict()
model = torch.load(g_model_path)
pretrained_dict = model.state_dict()
print("pretrained_dict type: ", type(pretrained_dict))
print("model : ",model)

pretrained_dict = {k: v for k,
                    v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
# 3. load the new state dict
generator.load_state_dict(model_dict)

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM

# Load the state dict from your custom checkpoint
checkpoint_path = "/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/train_checkpoint_2.pt"
checkpoint = torch.load(checkpoint_path)
checkpoint_state_dict = checkpoint['generator_state_dict']

# Assuming you know the configuration of your model (e.g., 'marian-mt-en-fr' for a Marian English to French model)
# Initialize the model with the same configuration
generator = AutoModelForSeq2SeqLM.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
generator_state_dict = generator.state_dict()

checkpoint_state_dict = {k.replace('module.', ''): v for k, v in checkpoint_state_dict.items() if k in generator_state_dict}

# Update the model's state dict with the loaded checkpoint
# This assumes 'generator_state_dict' is the key in your checkpoint dict containing the model's state dictionary
generator.load_state_dict(checkpoint_state_dict)

In [35]:
print("Checkpoint keys:", checkpoint_state_dict.keys())
print("Model keys:", generator_state_dict.keys())


Checkpoint keys: dict_keys([])
Model keys: odict_keys(['final_logits_bias', 'model.shared.weight', 'model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.encoder.layers.0.self_attn.k_proj.weight', 'model.encoder.layers.0.self_attn.k_proj.bias', 'model.encoder.layers.0.self_attn.v_proj.weight', 'model.encoder.layers.0.self_attn.v_proj.bias', 'model.encoder.layers.0.self_attn.q_proj.weight', 'model.encoder.layers.0.self_attn.q_proj.bias', 'model.encoder.layers.0.self_attn.out_proj.weight', 'model.encoder.layers.0.self_attn.out_proj.bias', 'model.encoder.layers.0.self_attn_layer_norm.weight', 'model.encoder.layers.0.self_attn_layer_norm.bias', 'model.encoder.layers.0.fc1.weight', 'model.encoder.layers.0.fc1.bias', 'model.encoder.layers.0.fc2.weight', 'model.encoder.layers.0.fc2.bias', 'model.encoder.layers.0.final_layer_norm.weight', 'model.encoder.layers.0.final_layer_norm.bias', 'model.encoder.layers.1.self_attn.k_proj.weight', 'model.encoder.layers.1.self_a

# use below

In [1]:
import torch
import dill

# Define the path to your saved model
# checkpoint_path_generator = "/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/best_generator.pt"
checkpoint_path_generator = '/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/train_checkpoint__generator2.pt'
# Load the entire model directly
generator2_train = torch.load(open(checkpoint_path_generator, "rb"), pickle_module=dill)

# Assuming you might also need to load the discriminator in a similar manner
checkpoint_path_discriminator = "/workspace/2024/Adversarial_NMT_th/checkpoints/bert_dualG/wmt14_en_fr_1mil_pg_kd_loss_MarianMT_1mil_20epochs/best_discriminator.pt"
discriminator_cnn = torch.load(open(checkpoint_path_discriminator, "rb"), pickle_module=dill)

# Now, generator2_train and discriminator_cnn are the loaded model objects and ready to use

In [2]:
# generator2_train # Extract the underlying model from the DataParallel wrapper
generator2_train = generator2_train.module if isinstance(generator2_train, torch.nn.DataParallel) else generator2_train

AttributeError: 'MarianMTModel' object has no attribute 'module'

In [3]:
generator2_train.state_dict().keys()

odict_keys(['final_logits_bias', 'model.shared.weight', 'model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.encoder.layers.0.self_attn.k_proj.weight', 'model.encoder.layers.0.self_attn.k_proj.bias', 'model.encoder.layers.0.self_attn.v_proj.weight', 'model.encoder.layers.0.self_attn.v_proj.bias', 'model.encoder.layers.0.self_attn.q_proj.weight', 'model.encoder.layers.0.self_attn.q_proj.bias', 'model.encoder.layers.0.self_attn.out_proj.weight', 'model.encoder.layers.0.self_attn.out_proj.bias', 'model.encoder.layers.0.self_attn_layer_norm.weight', 'model.encoder.layers.0.self_attn_layer_norm.bias', 'model.encoder.layers.0.fc1.weight', 'model.encoder.layers.0.fc1.bias', 'model.encoder.layers.0.fc2.weight', 'model.encoder.layers.0.fc2.bias', 'model.encoder.layers.0.final_layer_norm.weight', 'model.encoder.layers.0.final_layer_norm.bias', 'model.encoder.layers.1.self_attn.k_proj.weight', 'model.encoder.layers.1.self_attn.k_proj.bias', 'model.encoder.layers.1.s

In [4]:
generator2_train

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(59514, 512, padding_idx=59513)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(59514, 512, padding_idx=59513)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
generator2_train.to('cpu')
translations_t5_base_finetuned_wmt14_1mil = []
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
# for text in tqdm(texts):
for idx, text in tqdm(enumerate(texts), desc="Translating", total=len(texts)):
    # print("english sentence: ", text)
    # print("Original French Sentence: ", labels[texts.index(text)])
    inputs_t5_base_finetuned_wmt14_1mil = tokenizer(text, return_tensors="pt").input_ids
    # outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    outputs_t5_base_finetuned_wmt14_1mil = generator2_train.generate(inputs_t5_base_finetuned_wmt14_1mil, max_length=60, num_beams=5, early_stopping=True)
    translation_t5_base_finetuned_wmt14_1mil = tokenizer.decode(outputs_t5_base_finetuned_wmt14_1mil[0], skip_special_tokens=True)
    # print("translated french sentences:",translation_t5_finetuned_wmt14_1mil)
    translations_t5_base_finetuned_wmt14_1mil.append(translation_t5_base_finetuned_wmt14_1mil)
    # print("\n")

# Specify the file path
file_path_t5_base_finetuned_wmt14_1mil = os.path.join(getpwd, "translated_french_by_MarinaMT_600sentsFT.txt")

# Open the file in write mode
with open(file_path_t5_base_finetuned_wmt14_1mil, "w") as file:
    # Write each translation to the file
    for translation in translations_t5_base_finetuned_wmt14_1mil:
        file.write(translation + "\n")

Translating: 100%|██████████| 3003/3003 [34:04<00:00,  1.47it/s]


In [33]:
labels

['Spectaculaire saut en "wingsuit" au-dessus de Bogota',
 "Le sportif Jhonathan Florez a sauté jeudi d'un hélicoptère au-dessus de Bogota, la capitale colombienne.",
 "Equipé d'un wingsuit (une combinaison munie d'ailes), il est passé à 160 km/h au-dessus du célèbre sanctuaire Monserrate, situé à plus de 3 000 mètres d'altitude, où de nombreux badauds s'étaient rassemblés pour observer son exploit.",
 'Une boîte noire dans votre voiture\xa0?',
 "Alors que les planificateurs du réseau routier des États-Unis ont du mal à trouver l'argent nécessaire pour réparer l'infrastructure autoroutière en décrépitude, nombreux sont ceux qui entrevoient une solution sous forme d'une petite boîte noire qui se fixe au-dessus du tableau de bord de votre voiture.",
 "Les appareils, qui enregistrent tous les miles parcourus par un automobiliste et transmettent les informations aux fonctionnaires, sont au centre d'une tentative controversée à Washington et dans les bureaux gouvernementaux de la planificati

In [14]:
result_t5_base_finetuned_wmt14_1mil = metric.compute(predictions=translations_t5_base_finetuned_wmt14_1mil, references=labels)
result_t5_base_finetuned_wmt14_1mil = {"bleu": result_t5_base_finetuned_wmt14_1mil["score"]}
result_t5_base_finetuned_wmt14_1mil

{'bleu': 0.017488055190711223}